## Activity 1 - Pre-processing, training and selection

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("../datasets/bank-full.csv",sep=";") 

data.head(10)

Unnamed: 0  age           job   marital  education default  balance  \
0           0   58    management   married   tertiary      no     2143   
1           1   44    technician    single  secondary      no       29   
2           2   33  entrepreneur   married  secondary      no        2   
3           3   47   blue-collar   married        NaN      no     1506   
4           4   33           NaN    single        NaN      no        1   
5           5   35    management   married   tertiary      no      231   
6           6   28    management    single   tertiary      no      447   
7           7   42  entrepreneur  divorced   tertiary     yes        2   
8           8   58       retired   married    primary      no      121   
9           9   43    technician    single  secondary      no      593   

  housing loan contact  day month  duration  campaign  pdays  previous  \
0     yes   no     NaN    5   may       261         1     -1         0   
1     yes   no     NaN    5   may       151         1     -1         0   
2     yes  yes     NaN    5   may        76         1     -1         0   
3     yes   no     NaN    5   may        92         1     -1         0   
4      no   no     NaN    5   may       198         1     -1         0   
5     yes   no     NaN    5   may       139         1     -1         0   
6     yes  yes     NaN    5   may       217         1     -1         0   
7     yes   no     NaN    5   may       380         1     -1         0   
8     yes   no     NaN    5   may        50         1     -1         0   
9     yes   no     NaN    5   may        55         1     -1         0   

  poutcome   y  
0      NaN  no  
1      NaN  no  
2      NaN  no  
3      NaN  no  
4      NaN  no  
5      NaN  no  
6      NaN  no  
7      NaN  no  
8      NaN  no  
9      NaN  no

In [3]:
data.isnull().sum()

Unnamed: 0        0
age               0
job             288
marital           0
education      1857
default           0
balance           0
housing           0
loan              0
contact       13020
day               0
month             0
duration          0
campaign          0
pdays             0
previous          0
poutcome      36959
y                 0
dtype: int64

In [4]:
data = data.drop(["contact","poutcome"], axis=1)

In [5]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()

features_to_convert = ["job","marital","default","housing","loan","month","y"]
for i in features_to_convert:
    data[i] = enc.fit_transform(data[i].astype('str'))

In [6]:
data['education'] = data['education'].fillna('unknown')
encoder = ['unknown','primary','secondary','tertiary']
for i, word in enumerate(encoder):
    data['education'] = data['education'].str.replace(word, str(i))
data['education'] = data['education'].astype('int64')
data.head()

Unnamed: 0  age  job  marital  education  default  balance  housing  loan  \
0           0   58    4        1          3        0     2143        1     0   
1           1   44   10        2          2        0       29        1     0   
2           2   33    2        1          2        0        2        1     1   
3           3   47    1        1          0        0     1506        1     0   
4           4   33    5        2          0        0        1        0     0   

   day  month  duration  campaign  pdays  previous  y  
0    5      8       261         1     -1         0  0  
1    5      8       151         1     -1         0  0  
2    5      8        76         1     -1         0  0  
3    5      8        92         1     -1         0  0  
4    5      8       198         1     -1         0  0

In [7]:
outliers = {}
for i in range(data.shape[1]):
    min_t = data[data.columns[i]].mean() - (3 * data[data.columns[i]].std())
    max_t = data[data.columns[i]].mean() + (3 * data[data.columns[i]].std())
    count = 0
    for j in data[data.columns[i]]:
        if j < min_t or j > max_t:
            count += 1
    outliers[data.columns[i]] = [count,data.shape[0]-count]
print(outliers)

{'Unnamed: 0': [0, 45211], 'age': [381, 44830], 'job': [0, 45211], 'marital': [0, 45211], 'education': [0, 45211], 'default': [815, 44396], 'balance': [745, 44466], 'housing': [0, 45211], 'loan': [0, 45211], 'day': [0, 45211], 'month': [0, 45211], 'duration': [963, 44248], 'campaign': [840, 44371], 'pdays': [1723, 43488], 'previous': [582, 44629], 'y': [0, 45211]}


In [8]:
X = data.drop("y", axis=1)
Y = data["y"]

In [9]:
from sklearn.model_selection import train_test_split
X_new, X_test, Y_new, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 0)
X_train, X_dev, Y_train, Y_dev = train_test_split(X_new, Y_new, test_size=0.25, random_state = 0)
print(X_train.shape, X_dev.shape, X_test.shape, Y_train.shape, Y_dev.shape, Y_test.shape)

(27126, 15) (9042, 15) (9043, 15) (27126,) (9042,) (9043,)


In [10]:
from sklearn.tree import DecisionTreeClassifier
model_tree = DecisionTreeClassifier(random_state = 100)
model_tree.fit(X_train, Y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=100,
            splitter='best')

In [11]:
from sklearn.neural_network import MLPClassifier
model_NN = MLPClassifier(random_state = 100)
model_NN.fit(X_train, Y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=100, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [12]:
from sklearn.metrics import precision_score
X_sets = [X_train, X_dev, X_test]
Y_sets = [Y_train, Y_dev, Y_test]

precision = []

for i in range(0,len(X_sets)):
    
    pred = model_tree.predict(X_sets[i])
    score = precision_score(Y_sets[i], pred)
    precision.append(score)

precision

[1.0, 0.5294117647058824, 0.4782608695652174]

In [13]:
from sklearn.metrics import precision_score
X_sets = [X_train, X_dev, X_test]
Y_sets = [Y_train, Y_dev, Y_test]

precision = []

for i in range(0,len(X_sets)):
    
    pred = model_NN.predict(X_sets[i])
    score = precision_score(Y_sets[i], pred)
    precision.append(score)

precision

[0.563150074294205, 0.5545454545454546, 0.5186915887850467]

In [14]:
from sklearn.tree import DecisionTreeClassifier
model_tree = DecisionTreeClassifier(random_state = 100, min_samples_leaf=100,max_depth=100)
model_tree.fit(X_train, Y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=100,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=100, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=100,
            splitter='best')

In [15]:
from sklearn.neural_network import MLPClassifier
model_NN = MLPClassifier(random_state = 100, max_iter=1000,  hidden_layer_sizes = [100,100,50,25,25], tol=1e-7)
model_NN.fit(X_train, Y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=[100, 100, 50, 25, 25], learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=100, shuffle=True, solver='adam', tol=1e-07,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [16]:
from sklearn.metrics import precision_score
X_sets = [X_train, X_dev, X_test]
Y_sets = [Y_train, Y_dev, Y_test]

precision = []

for i in range(0,len(X_sets)):
    
    pred = model_tree.predict(X_sets[i])
    score = precision_score(Y_sets[i], pred)
    precision.append(score)

precision

[0.6504424778761062, 0.6569264069264069, 0.6016949152542372]

In [17]:
from sklearn.metrics import precision_score
X_sets = [X_train, X_dev, X_test]
Y_sets = [Y_train, Y_dev, Y_test]

precision = []

for i in range(0,len(X_sets)):
    
    pred = model_NN.predict(X_sets[i])
    score = precision_score(Y_sets[i], pred)
    precision.append(score)

precision

[0.6264236902050114, 0.6123128119800333, 0.5806451612903226]

## Activity 2 - Save the model

In [18]:
import pickle
import os

In [19]:
path = os.getcwd()+'/final_model.pkl'
file = open(path, 'wb')
pickle.dump(model_NN, file)